In [2]:
import os
import uuid
import shutil
#import pylatex
import pandas as pd
import lightkurve as lk 
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

from datetime import datetime
from tqdm.notebook import tqdm
from multiprocessing import cpu_count, Pool
#from pylatex.section import Chapter, Subsection
#from pylatex import Document, Section, Figure, NoEscape, Command

%run pipeline.ipynb

import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', None)

pop 106 and 110

In [4]:
coltypes={'ID':str,'TIC':str,'gaiadr3_source_id':str,'epic_id':str,'Score1':'Int64','Flare1':'Int64',
          'Shelf1':'Int64','Evolution1':'Int64','Score2':'Int64','Flare2':'Int64','Shelf2':'Int64','Evolution2':'Int64'}

In [5]:
lctable=pd.read_csv('lcscoresandchecker.csv',dtype=coltypes)

In [6]:
lctable110=lctable[lctable.pop_id==110]

In [7]:
#lctable110

Group 1: 1-13, Group 2: 27-39, Group 3: 61-69, Group 4: 87-90, Group 5: 93-96

In [9]:
#lc110.secorcamp

In [10]:
#lc110[lc110.Shelf1==1]

In [11]:
#search_result110 = lk.search_lightcurve("TIC 294328887", author="SPOC", cadence=120)

In [12]:
#spoclcs110=search_result110.download_all()

In [13]:
# lc1=get_lc_string('TIC-294328887-1-FFI-30min')
# lc2=get_lc_string('TIC-294328887-2-FFI-30min')
# lclist=[lc1,lc2]

In [14]:
#spoclcs110.append(lc1)

In [15]:
#spoclcs110

In [16]:
#lk.LightCurveCollection(lclist)

In [17]:
# lclist110_1=[]
# lctable110_1=lctable110[np.logical_and(lctable110.secorcamp>=6,lctable110.secorcamp<=13)]
# for index, row in lctable110_1.iterrows():
#     curlc=get_lc_string(row.lcname)
#     if row.LC_author=='FFI':
#         curlc.flux=curlc.flux+1
#     elif row.LC_author=='SPOC':
#         curlc=curlc.normalize()
#     lclist110_1.append(curlc)

In [18]:
#lccoll110_1=lk.LightCurveCollection(lclist110_1)

In [19]:
#lccoll110_1

In [20]:
#lcstitch110_1=lccoll110_1.stitch()

In [21]:
# fig=subgraph_rp(lcstitch110_1,period=u.Quantity(0.35458333,u.d))
# fig.savefig('multiplots/riverplots/110_sec6_13.png',bbox_inches='tight')

In [22]:
def megariver(pop_id,secs,ax=None,epoch_time=None,vmin=None,vmax=None):
    startsec=secs[0]
    endsec=secs[1]
    table=lctable[np.logical_and(lctable.pop_id==pop_id,
                                 np.logical_and(lctable.secorcamp>=startsec,lctable.secorcamp<=endsec))]
    lclist=[]
    for index, row in table.iterrows():
        curlc=get_lc_string(row.lcname)
        if row.LC_author=='FFI':
            curlc.flux=curlc.flux+1
        elif row.LC_author=='SPOC':
            curlc=curlc.normalize()
        lclist.append(curlc)
    lccoll=lk.LightCurveCollection(lclist)
    lcstitch=lccoll.stitch()
    period=u.Quantity(table.per.values[0],u.d)
    if ax==None:
        fig, epoch_time, vmin, vmax, im=mega_rp(lcstitch,period=period,epoch_time=epoch_time,vmin=vmin,vmax=vmax)
        fig.savefig(f'multiplots/riverplots/{pop_id}_sec{startsec}_{endsec}.png',bbox_inches='tight')
    else:
        epoch_time, vmin, vmax, im=mega_rp(lcstitch,period=period,ax=ax,epoch_time=epoch_time,vmin=vmin,vmax=vmax)
    return epoch_time, vmin, vmax, im

In [23]:
def supermegariver(pop_id,seclist): #sec list should be a tuple of 2 element tuples and must have at least 2
    numplots=len(seclist)
    fig, axes = plt.subplots(1, numplots,figsize=(6*numplots,12))

    #first stitch them 

    #plot first mega river to get epoch_time and vmin and vmax
    #not a great way to handle it because the range could be wider on the second one
    #but it's close enough here
    epoch_time, vmin, vmax, im=megariver(pop_id,seclist[0],ax=axes[0])
    print(vmin,vmax)

    for i in range(1, len(axes)):
        megariver(pop_id,seclist[i],ax=axes[i],epoch_time=epoch_time,vmin=vmin,vmax=vmax)

    #divider = make_axes_locatable(ax)
    #cax = divider.append_axes("right", size="3%", pad="1%")
    cbar = fig.colorbar(im, ax=axes)
    cbar.set_label("Normalized Flux")

    fig.savefig(f'multiplots/riverplots/{pop_id}_supermega_sec{seclist[0][0]}_{seclist[-1][-1]}.png')

In [24]:
#epoch_time=megariver(110,(87,90))

In [25]:
#megariver(110,(93,96),epoch_time=epoch_time)

In [26]:
supermegariver(110,((87,90),(93,96)))

None None
0.9544755607843399 1.045562705397606
0.9544755607843399 1.045562705397606
0.9544755607843399 1.045562705397606
0.9544755607843399 1.045562705397606
